In [1]:
pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import urllib.request 
from mip import Model, xsum, maximize, BINARY, LinExpr

In [3]:
def CTFPFunction(graphUrl, skillsUrl, numTeams, teamUsersProportion, graphSep = "\n", numUsersSep = " ", skillsSep = "\n", numSkillsSep = " "):

  #1 - CRIANDO MATRIZ DE ADJACÊNCIA DAS RELAÇÕES ENTRE OS USUÁRIOS
  graph = urllib.request.urlopen(graphUrl).read().decode('utf-8').split(graphSep)
  del graph[-1] #vazio

  numUsers = int(graph[0].split(numUsersSep)[0])
  relationships = np.zeros(numUsers*numUsers, dtype=np.int64).reshape(numUsers,numUsers)

  for line in graph[1:]:
    l = [x for x in line.split(" ") if x]
    l[0] = int(l[0])
    l[1] = int(l[1])
    l[2] = round(float(l[2]))
    if int(l[0]) < numUsers and int(l[1]) < numUsers:
      relationships[l[0],l[1]] = relationships[l[1],l[0]] = int(l[2])

  print(f'Número usuários: {numUsers}')
  print(f'Matriz de relacionamentos {relationships}')
  print('\n')


  #2 - DEFININDO CONJUNTO DE SKILLS DE CADA USUÁRIO
  skillsData = urllib.request.urlopen(skillsUrl).read().decode('utf-8').split(skillsSep)
  numSkills = int(skillsData[0].split(numSkillsSep)[0])

  print(f'Número de skills: {numSkills}')
  print(f'Lista de skills: {skillsData[0].split(numSkillsSep)[1:]}')
  print('\n')


  #3 - CONTANDO O NÚMERO DE USUÁRIOS PARA CADA SKILL
  usersSkillsMatrix = np.zeros(numUsers*numSkills, dtype=np.int64).reshape(numUsers,numSkills)

  for index,item in enumerate(skillsData[1:]):
    usersSkillsMatrix[index] = np.asarray(item.split(" "))

  numUsersBySkill = usersSkillsMatrix.sum(axis=0)


  #4 - FUNÇÃO QUE RETORNA O CONJUNTO DE SKILLS DE UM USUÁRIO
  def userSkillSet(user):
    return np.asarray([id for id,s in enumerate(skillsData[user+1].split(" ")) if s and int(s) == 1])

  
  #5 - DEFININDO DEMANDA DE USUÁRIOS POR SKILL PARA CADA TIME
  teamDemand = np.full((numTeams, numSkills), 1, dtype=np.int64)

  for index,item in enumerate(numUsersBySkill):
    teamDemand[0][index] = round(item/teamUsersProportion)
    teamDemand[1][index] = round(item/teamUsersProportion)

  print(f'Demanda de cada time: {teamDemand}')
  print('\n')

  #6 - CONJUNTO DE SKILLS QUE CADA TIME PRECISA
  def demandSkillSet(team):
    return np.asarray([id for id,s in enumerate(teamDemand[team]) if s > 0])


  #7 - IMPLEMENTAÇÃO DO MODELO
  model = Model("CTFP")

  #7.1 - Variáveis
  x = [[[model.add_var(var_type=BINARY) for s in range(numSkills)] for u in range(numUsers)] for t in range(numTeams)]

  #7.2 - Função objetivo
  model.objective = maximize(xsum( xsum( xsum( x[t][u][s] for s in userSkillSet(u) ) for u in range(numUsers) ) for t in range(numTeams)))

  #7.3 - Restrições

  #restrição (63)
  for t in range(numTeams):
    for u in range(numUsers):
      for v in range(numUsers):
        if relationships[u,v] == -1:
          model += xsum(x[t][u][s] for s in userSkillSet(u)) + xsum(x[t][v][s] for s in userSkillSet(v)) <= 1

  #restrição (64)
  for t1 in range(numTeams):
    for t2 in np.arange(t1+1, numTeams, 1):
      for u in range(numUsers):
        for v in range(numUsers):
          if relationships[u,v] == 1:
            model += xsum(x[t1][u][s] for s in userSkillSet(u)) + xsum(x[t2][v][s] for s in userSkillSet(v)) <= 1

  #restrição (65)
  for u in range(numUsers):
    model += xsum( xsum( x[t][u][s] for s in userSkillSet(u)) for t in range(numTeams)) <= 1

  #restrição (66)
  for t in range(numTeams):
    for s in demandSkillSet(t):
      model += xsum( x[t][u][s] for u in range(numUsers) if s in userSkillSet(u)) >= teamDemand[t,s]

  model.optimize()

  print('\n\n')

  for teamIdx,team in enumerate(x):
    usersCount = 0
    skillsUsersCount = [0] * numSkills
    for userIdx,user in enumerate(team):
      for skillIdx,skill in enumerate(user):
        skillsUsersCount[skillIdx] += skill.x
        usersCount += skill.x
    
    print(f'Análise da equipe {teamIdx}')

    skillsUsersCountStr = ' | '.join(str(ind)+': '+str(s) for ind, s in enumerate(skillsUsersCount))
    print(f'Número de indivíduos na equipe: {usersCount}')
    print(f'Número de indíviduos em cada skill: {skillsUsersCountStr}')

    print('\n')

  print('\n\n')

  print(f'Número de indivíduos alocados: {model.objective_value}')

In [4]:
CTFPFunction('https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/congress-2010-v1.g',
             'https://raw.githubusercontent.com/natalialidia/analise-parlamentos/main/brcongress-instances/state-as-skill/2010.txt',
             2,
             5,
             numUsersSep="\t")

Número usuários: 545
Matriz de relacionamentos [[0 1 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]
 [1 1 0 ... 0 1 1]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]]
Número de skills: 27
Demanda de cada time: [[11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]
 [11 14 10  9  5  2  2  6  4  6  2  2  2  5  2  3  2  4  2  2  2  2  4  2
   2  2  2]]


Análise da equipe 0
Número de indivíduos na equipe: 157.0
Número de indíviduos em cada skill: 0: 14.0 | 1: 23.0 | 2: 19.0 | 3: 11.0 | 4: 7.0 | 5: 2.0 | 6: 4.0 | 7: 10.0 | 8: 4.0 | 9: 8.0 | 10: 2.0 | 11: 6.0 | 12: 3.0 | 13: 7.0 | 14: 2.0 | 15: 3.0 | 16: 3.0 | 17: 4.0 | 18: 2.0 | 19: 2.0 | 20: 2.0 | 21: 2.0 | 22: 7.0 | 23: 3.0 | 24: 3.0 | 25: 2.0 | 26: 2.0
Análise da equipe 1
Número de indivíduos na equipe: 174.0
Número de indíviduos em cada skill: 0: 21.0 | 1: 26.0 | 2: 19.0 | 3: 15.0 | 4: 8.0 | 5: 2.0 | 6: 4.0 | 7: 7.0 | 8: 4.0 | 9: 6.0 | 10: 2.0 | 11: 2.0 | 12: 3.0 | 13: 8.0 | 14: 2.0 | 15: 4.0 | 16: 4.0 | 